In [1]:
from pandas import read_csv, DataFrame
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score, roc_auc_score, roc_curve
from sklearn.cross_validation import train_test_split
import matplotlib.pyplot as plt


learn = read_csv('full_learn_dataset.csv',';',low_memory=False, header =0, )
learn = learn.fillna(0)
fraud = {'fraud': 1, 'good': 0}
flag = {1: True, 0: False}

learn['type'] = learn['type'].replace(fraud)
learn['BK_PIN_FLAG'] = learn['BK_PIN_FLAG'].replace(flag)
learn['STORNO_APPROVE_FLAG'] = learn['STORNO_APPROVE_FLAG'].replace(flag)
learn['MAIN_OPERATION'] = learn['MAIN_OPERATION'].replace(flag)

trg = learn['type']
trn = learn.drop(['type','id','evt_type','FLAG_ID_STORNO_APPROVE','FLAG_BK_PIN'], axis=1)

In [2]:
Xtrn, Xtest, Ytrn, Ytest = train_test_split(trn, trg, test_size=0.3)

In [15]:
model = LogisticRegression() # логистическая регрессия

model.fit(Xtrn, Ytrn)
roc = roc_auc_score(Ytest, model.predict(Xtest))
fpr, tpr, thresholds  = roc_curve(Ytest, model.predict(Xtest))

### Feature Selection

In [7]:
from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import RFE

In [8]:
logreg = LogisticRegression(penalty='l2',class_weight=)
rfe = RFE(logreg, 5)
selector = rfe.fit(Xtrn,Ytrn)

In [26]:
trn.columns[np.where(selector.ranking_ == 1)]

Index([u'1115_post', u'1164_post', u'6071542409_post', u'6101688611_post',
       u'7163029317_prev'],
      dtype='object')

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline  
plt.plot(fpr, tpr)

In [25]:
model.coef_[0][np.where(selector.ranking_ == 1)]

array([ 0.94836724, -0.61985865,  0.62991553,  1.42579973,  0.82809979])

## Data Grid on weights

In [27]:
features,labels = Xtrn, Ytrn

In [42]:
from sklearn import tree
from sklearn import grid_search

In [46]:
clf1 = tree.DecisionTreeClassifier()
parameters_to_tune = {
                        'min_samples_split':[2,4,6,10,15,25], 'min_samples_leaf':[1,2,4,10],'max_depth':[None,4,10,15],
                        'splitter' : ('best','random'),'max_features':[None,2,4,6,8,10,12,14],'class_weight':[{1:10},"auto"]
                     }
clf=grid_search.GridSearchCV(clf1,parameters_to_tune)
clf.fit(features,labels)
print clf.best_params_

{'splitter': 'best', 'min_samples_leaf': 1, 'min_samples_split': 25, 'max_features': 2, 'max_depth': 4, 'class_weight': {1: 10}}


In [48]:
tree.export_graphviz(
    clf.best_estimator_.tree_,
    out_file='gridsearch_best-tree.dot'
)

In [50]:
clf2 = tree.DecisionTreeClassifier()
parameters_to_tune = {
                        'min_samples_leaf':[1,2,4,10],'max_depth':[None,4,10],
                        'splitter' : ('best','random'),'max_features':[4,6,10],'class_weight':[{1:10},{1:20},{1:50},{1:100},"auto"]
                     }
my_clf=grid_search.GridSearchCV(clf2,parameters_to_tune)
my_clf.fit(features,labels)
print my_clf.best_params_

{'max_features': 4, 'splitter': 'random', 'min_samples_leaf': 10, 'max_depth': None, 'class_weight': {1: 10}}


## Cross Validation Scoring - Recall

In [59]:
?tree.DecisionTreeClassifier

In [60]:
clf3 = tree.DecisionTreeClassifier()
parameters_to_tune = {
                        'min_samples_leaf':[1,2,4,10],'max_depth':[None,4,10],
                        'splitter' : ('best','random'),'max_features':[4,6,10],'class_weight':[{1:10},{1:20},{1:50},{1:100},"auto"]
                     }
my_clf3=grid_search.GridSearchCV(clf3,parameters_to_tune,scoring='recall')
my_clf3.fit(features,labels)
print my_clf3.best_params_

{'max_features': 6, 'splitter': 'random', 'min_samples_leaf': 4, 'max_depth': 4, 'class_weight': 'auto'}


In [62]:
tree.export_graphviz(
    my_clf3.best_estimator_.tree_,
    out_file='3.5_gridsearch_recall-tree.dot'
)

In [63]:
my_clf3.best_estimator_

DecisionTreeClassifier(class_weight='auto', criterion='gini', max_depth=4,
            max_features=6, max_leaf_nodes=None, min_samples_leaf=4,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            random_state=None, splitter='random')